In [1]:
import pandas as pd
import plotly.express as px
#import hvplot.pandas
from sklearn.cluster import KMeans

In [2]:
# Loading data
file_path = "sources/bigfoot_sightings.csv"
df_bigfoot = pd.read_csv(file_path)
df_bigfoot.head(10)

,Unnamed: 0,number,classification,timestamp,latitude,longitude,report,key_words
0,0,637,Class A,2000-06-16T12:00:00Z,61.50000,-142.90000,Campers' encounter just after dark in the Wra...,Campers' encounter dark Wrangell - St. Elias N...
1,1,2917,Class A,1995-05-15T12:00:00Z,55.18720,-132.79820,Family observes large biped from car,Family observes large biped car
2,2,7963,Class A,2004-02-09T12:00:00Z,55.20350,-132.82020,Sasquatch walks past window of house at night,Sasquatch walks past window house night
3,3,9317,Class A,2004-06-18T12:00:00Z,62.93750,-141.56670,"Driver on Alcan Highway has noon, road encoun...","Driver Alcan Highway noon, road encounter near..."
4,4,13038,Class A,2004-02-15T12:00:00Z,61.05950,-149.78530,Snowmobiler has encounter in deep snow near P...,"Snowmobiler encounter deep snow near Potter, AK"
5,5,23666,Class A,2008-04-23T12:00:00Z,62.77335,-141.31650,Passenger on the Alcan Highway sees a tall bi...,Passenger Alcan Highway sees tall bipedal crea...
6,6,26604,Class A,2009-07-15T12:00:00Z,64.89139,-147.81420,Daytime sighting of reddish-colored bigfoot n...,Daytime sighting reddish-colored bigfoot near ...
7,7,179,Class A,1981-09-15T12:00:00Z,32.31435,-85.16235,"Man and witnesses have two seperate sighting,...","Man witnesses two seperate sighting, 1st fishi..."
8,8,245,Class A,1999-07-15T12:00:00Z,33.28375,-87.32655,"Two outdoorsman fishing, loud vocalizations, ...","Two outdoorsman fishing, loud vocalizations, f..."
9,9,416,Class A,1983-11-15T12:00:00Z,34.95605,-86.45590,A resident's early evening sighting next to h...,A resident's early evening sighting next home ...


In [3]:
df_bigfoot.columns.tolist()

['Unnamed: 0',
 'number',
 'classification',
 'timestamp',
 'latitude',
 'longitude',
 'report',
 'key_words']

In [4]:
skeleton_df = df_bigfoot.drop(['Unnamed: 0','number', 'report','key_words'], axis=1)
skeleton_df.head()

,classification,timestamp,latitude,longitude
0,Class A,2000-06-16T12:00:00Z,61.5000,-142.9000
1,Class A,1995-05-15T12:00:00Z,55.1872,-132.7982
2,Class A,2004-02-09T12:00:00Z,55.2035,-132.8202
3,Class A,2004-06-18T12:00:00Z,62.9375,-141.5667
4,Class A,2004-02-15T12:00:00Z,61.0595,-149.7853


In [5]:
#List the data types
skeleton_df.dtypes

classification     object
timestamp          object
latitude          float64
longitude         float64
dtype: object

In [6]:
import datetime

#skeleton_df['date'] = skeleton_df['timestamp'].apply(lambda d: datetime.date.fromtimestamp(d))
#skeleton_df.head()

skeleton_df['timestamp'] = pd.to_datetime(skeleton_df['timestamp'])
skeleton_df.head()

,classification,timestamp,latitude,longitude
0,Class A,2000-06-16 12:00:00+00:00,61.5000,-142.9000
1,Class A,1995-05-15 12:00:00+00:00,55.1872,-132.7982
2,Class A,2004-02-09 12:00:00+00:00,55.2035,-132.8202
3,Class A,2004-06-18 12:00:00+00:00,62.9375,-141.5667
4,Class A,2004-02-15 12:00:00+00:00,61.0595,-149.7853


In [7]:
#List the data types
skeleton_df.dtypes

classification                 object
timestamp         datetime64[ns, UTC]
latitude                      float64
longitude                     float64
dtype: object

In [12]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
skeleton_df["classification"] = label_encoder.fit_transform(skeleton_df["classification"])
skeleton_df.head()

,classification,timestamp,latitude,longitude
0,0,2000-06-16 12:00:00+00:00,61.5000,-142.9000
1,0,1995-05-15 12:00:00+00:00,55.1872,-132.7982
2,0,2004-02-09 12:00:00+00:00,55.2035,-132.8202
3,0,2004-06-18 12:00:00+00:00,62.9375,-141.5667
4,0,2004-02-15 12:00:00+00:00,61.0595,-149.7853


In [13]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
data_scaler = StandardScaler()


In [15]:
skeleton_df = skeleton_df.drop(['timestamp'], axis=1)
skeleton_df.head()

,classification,latitude,longitude
0,0,61.5000,-142.9000
1,0,55.1872,-132.7982
2,0,55.2035,-132.8202
3,0,62.9375,-141.5667
4,0,61.0595,-149.7853


In [16]:
# Generate our categorical variable list
skeleton_scaled = data_scaler.fit_transform(skeleton_df)


In [17]:
from sklearn.preprocessing import StandardScaler
data_scaler = StandardScaler()

In [18]:
skeleton_scaled[:5]


array([[-0.99566935,  3.58381627,  0.03125306],
       [-0.99566935,  2.54305752,  0.03125801],
       [-0.99566935,  2.54574482,  0.031258  ],
       [-0.99566935,  3.82080947,  0.03125371],
       [-0.99566935,  3.5111933 ,  0.03124968]])

In [19]:
import numpy as np
print(np.mean(skeleton_scaled[:,0]))
print(np.std(skeleton_scaled[:,0]))


0.0
1.0


In [20]:
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


In [21]:
# Define the features set.
X = skeleton_df.copy()
X = X.drop("classification", axis=1)
X.head()

,latitude,longitude
0,61.5000,-142.9000
1,55.1872,-132.7982
2,55.2035,-132.8202
3,62.9375,-141.5667
4,61.0595,-149.7853


In [22]:
# Define the target set.
y = skeleton_df["classification"].values
y[:5]

array([0, 0, 0, 0, 0])

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)


In [24]:
# Determine the shape of our training and testing sets.
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2857, 2)
(953, 2)
(2857,)
(953,)


In [25]:
# Splitting into Train and Test sets into an 80/20 split.
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, random_state=78, train_size=0.80)


In [26]:
# Determine the shape of our training and testing sets.
print(X_train2.shape)
print(X_test2.shape)
print(y_train2.shape)
print(y_test2.shape)

(3048, 2)
(762, 2)
(3048,)
(762,)


In [27]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [28]:
# Creating the decision tree classifier instance.
model = tree.DecisionTreeClassifier()
# Fitting the model.
model = model.fit(X_train_scaled, y_train)

In [29]:
# Making predictions using the testing data.
predictions = model.predict(X_test_scaled)

In [30]:
predictions

array([0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1,
       1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1,

In [35]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm

array([[258, 235,   1],
       [224, 228,   0],
       [  4,   3,   0]])

In [38]:

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1", "Actual 2"], columns=["Predicted 0", "Predicted 1", "Predicted 2"])

cm_df

,Predicted 0,Predicted 1,Predicted 2
Actual 0,258,235,1
Actual 1,224,228,0
Actual 2,4,3,0


In [39]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [40]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1,Predicted 2
Actual 0,258,235,1
Actual 1,224,228,0
Actual 2,4,3,0


Accuracy Score : 0.5099685204616999
Classification Report
              precision    recall  f1-score   support

           0       0.53      0.52      0.53       494
           1       0.49      0.50      0.50       452
           2       0.00      0.00      0.00         7

    accuracy                           0.51       953
   macro avg       0.34      0.34      0.34       953
weighted avg       0.51      0.51      0.51       953



In [41]:
from sklearn.ensemble import RandomForestClassifier


In [42]:
# Define the features set.
X = skeleton_df.copy()
X = X.drop("classification", axis=1)
X.head()

,latitude,longitude
0,61.5000,-142.9000
1,55.1872,-132.7982
2,55.2035,-132.8202
3,62.9375,-141.5667
4,61.0595,-149.7853


In [43]:
# Define the target set.
y = skeleton_df["classification"].values
y[:5]

array([0, 0, 0, 0, 0])

In [44]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [45]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [52]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=500, random_state=78) 

In [53]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [54]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [55]:
predictions

array([0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1,
       1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1,
       1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0,

In [56]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1", "Actual 2"], columns=["Predicted 0", "Predicted 1", "Predicted 2"])

cm_df

,Predicted 0,Predicted 1,Predicted 2
Actual 0,272,222,0
Actual 1,213,239,0
Actual 2,4,3,0


In [57]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [58]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1,Predicted 2
Actual 0,272,222,0
Actual 1,213,239,0
Actual 2,4,3,0


Accuracy Score : 0.5362014690451207
Classification Report
              precision    recall  f1-score   support

           0       0.56      0.55      0.55       494
           1       0.52      0.53      0.52       452
           2       0.00      0.00      0.00         7

    accuracy                           0.54       953
   macro avg       0.36      0.36      0.36       953
weighted avg       0.53      0.54      0.53       953



/opt/anaconda3/envs/c19/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/c19/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/c19/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [59]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances


array([0.67689345, 0.32310655])

In [60]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.6768934525847955, 'latitude'), (0.32310654741520445, 'longitude')]

In [65]:
import joblib


In [67]:
# save
joblib.dump(RandomForestClassifier, "./random_forest.joblib")


['./random_forest.joblib']

In [68]:
# load, no need to initialize the loaded_rf
loaded_rf = joblib.load("./random_forest.joblib")